In [2]:
import numpy as np
import random

class ConnectFour:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.done = False
        self.winner = None

    def reset(self):
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.done = False
        self.winner = None
        return self.board

    def available_actions(self):
        return [c for c in range(self.cols) if self.board[0][c] == 0]

    def step(self, action, player):
        # Drop piece to the lowest empty row in the chosen column
        for r in reversed(range(self.rows)):
            if self.board[r][action] == 0:
                self.board[r][action] = player
                break

        reward = 0
        if self.check_win(player):
            self.done = True
            self.winner = player
            reward = 1
        elif len(self.available_actions()) == 0:
            self.done = True
            self.winner = 0  # tie
            reward = 0
        return self.board, reward, self.done

    def check_win(self, player):
        # Check horizontal
        for r in range(self.rows):
            for c in range(self.cols - 3):
                if all(self.board[r, c+i] == player for i in range(4)):
                    return True
        # Check vertical
        for r in range(self.rows - 3):
            for c in range(self.cols):
                if all(self.board[r+i, c] == player for i in range(4)):
                    return True
        # Check diagonal /
        for r in range(3, self.rows):
            for c in range(self.cols - 3):
                if all(self.board[r-i, c+i] == player for i in range(4)):
                    return True
        # Check diagonal \
        for r in range(self.rows - 3):
            for c in range(self.cols - 3):
                if all(self.board[r+i, c+i] == player for i in range(4)):
                    return True
        return False

    def render(self):
        # Display the board with emojis for clarity
        symbol_map = {0: "⚪", 1: "🔴", 2: "🟡"}
        for r in range(self.rows):
            print(" ".join(symbol_map[cell] for cell in self.board[r]))
        print("\n0 1 2 3 4 5 6\n")  # Column numbers for reference


class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}  # store Q-values as dict to handle large states

    def get_state_key(self, state):
        return tuple(state.flatten())

    def choose_action(self, state):
        actions = self.env.available_actions()
        if random.random() < self.epsilon:
            return random.choice(actions)
        # Exploit: pick action with max Q
        state_key = self.get_state_key(state)
        q_values = [self.q_table.get((state_key, a), 0) for a in actions]
        max_q = max(q_values)
        max_actions = [a for a, q in zip(actions, q_values) if q == max_q]
        return random.choice(max_actions)

    def update_q(self, state, action, reward, next_state, done):
        state_key = self.get_state_key(state)
        next_key = self.get_state_key(next_state)
        max_next_q = 0
        if not done:
            actions = self.env.available_actions()
            max_next_q = max([self.q_table.get((next_key, a), 0) for a in actions])
        old_q = self.q_table.get((state_key, action), 0)
        self.q_table[(state_key, action)] = old_q + self.alpha * (reward + self.gamma * max_next_q - old_q)

    def train(self, episodes=5000):
        for _ in range(episodes):
            state = self.env.reset()
            done = False
            player = 1
            while not done:
                action = self.choose_action(state)
                next_state, reward, done = self.env.step(action, player)
                self.update_q(state, action, reward, next_state, done)
                state = next_state
                player = 2 if player == 1 else 1


# Initialize environment and agent
env = ConnectFour()
agent = QLearningAgent(env)
agent.train(episodes=5000)

# Play against human
state = env.reset()
env.render()
while not env.done:
    # Human move
    valid = False
    while not valid:
        try:
            col = int(input("Your move (0-6): "))
            if col in env.available_actions():
                valid = True
            else:
                print("Column full or invalid. Try again.")
        except:
            print("Invalid input. Enter 0-6.")
    state, _, done = env.step(col, 1)
    env.render()
    if env.done:
        break

    # AI move
    ai_action = agent.choose_action(state)
    print(f"AI chooses column {ai_action}")
    state, _, done = env.step(ai_action, 2)
    env.render()

# Print result
if env.winner == 1:
    print("You win! 🎉")
elif env.winner == 2:
    print("AI wins! 🤖")
else:
    print("It's a tie! 😐")


⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪

0 1 2 3 4 5 6



Your move (0-6):  0


⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ ⚪

0 1 2 3 4 5 6

AI chooses column 6
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ 🟡

0 1 2 3 4 5 6



Your move (0-6):  6


⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ 🔴
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ 🟡

0 1 2 3 4 5 6

AI chooses column 4
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ 🔴
🔴 ⚪ ⚪ ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6



...

⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ 🔴
🔴 ⚪ ⚪ ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6

AI chooses column 2
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ 🔴
🔴 ⚪ 🟡 ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6



Your move (0-6):  0


⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ 🔴
🔴 ⚪ 🟡 ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6

AI chooses column 4
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ 🟡 ⚪ 🔴
🔴 ⚪ 🟡 ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6



Your move (0-6):  0


⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
⚪ ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ ⚪ ⚪ ⚪
🔴 ⚪ ⚪ ⚪ 🟡 ⚪ 🔴
🔴 ⚪ 🟡 ⚪ 🟡 ⚪ 🟡

0 1 2 3 4 5 6

You win! 🎉
